In [1]:
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

import matplotlib.pyplot as plt

import os
import sys

from IPython.display import display, Image

from scipy import ndimage

import random
import cv2 
from sklearn.model_selection import train_test_split
%matplotlib inline

def show_img(img):
    fig = plt.gcf()
    fig.set_size_inches(18.5, 10.5)
    if len(img.shape) == 3:
        plt.imshow(img)
    else:
        img2 = img
        plt.imshow(img2, cmap='gray')
        plt.show()

# Get three channel data

In [2]:
%load_ext autoreload
%autoreload 2
from utils.preprocess_utils import *

In [3]:
shape = (64, 64)
channel = 3
data = generate_data_set_for_training(shape, channel)

dataset: (86345, 64, 64, 3)
labels: (86345, 6)
dataset: (13066, 64, 64, 3)
labels: (13066, 6)


In [4]:
X_train = data['X_train']
X_val = data['X_val']
y_train = data['y_train']
y_val = data['y_val']
X_test = data['X_test']
y_test = data['y_test']

# Try pretrianed Vgg

In [5]:
import numpy as np
import os
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Activation,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras import regularizers
import keras.utils as ku
from keras import backend as K
from keras import optimizers
import tensorflow as tf
import pickle
import matplotlib.pyplot as ply

Using TensorFlow backend.


In [7]:
from utils.vgg16_utils import *

In [8]:
# from scratch
run_pretrained_vgg16()

dataset: (86345, 64, 64, 3)
labels: (86345, 6)
dataset: (13066, 64, 64, 3)
labels: (13066, 6)
Train on 69076 samples, validate on 17269 samples
Epoch 1/50
69076/69076 [==============================] - 88s 1ms/step - loss: 25.1023 - num_loss: 6.2045 - dig1_loss: 6.1920 - dig2_loss: 4.6267 - dig3_loss: 1.6416 - dig4_loss: 0.2354 - nC_loss: 6.2021 - num_acc: 0.6134 - dig1_acc: 0.6142 - dig2_acc: 0.7115 - dig3_acc: 0.8972 - dig4_acc: 0.9848 - nC_acc: 0.6142 - val_loss: 25.3144 - val_num_loss: 6.2563 - val_dig1_loss: 6.2423 - val_dig2_loss: 4.6640 - val_dig3_loss: 1.6782 - val_dig4_loss: 0.2175 - val_nC_loss: 6.2563 - val_num_acc: 0.6118 - val_dig1_acc: 0.6127 - val_dig2_acc: 0.7106 - val_dig3_acc: 0.8959 - val_dig4_acc: 0.9865 - val_nC_acc: 0.6118

Epoch 00001: loss improved from inf to 25.10234, saving model to saved_models/VGGPreTrained.classifier.hdf5
Epoch 2/50
58944/69076 [========================>.....] - ETA: 11s - loss: 25.1449 - num_loss: 6.2163 - dig1_loss: 6.1991 - dig2_loss: 4

KeyboardInterrupt: 

In [114]:
_, row, col, channel = X_train.shape
print('chanel', channel)
print('row', row)
print('col', col)

chanel 3
row 64
col 64


In [115]:
# Params
digLen = 5
numDigits = 11

In [117]:
preTrainModel = VGG16(include_top = False, weights = 'imagenet')
preTrainModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [118]:
ptInput = keras.Input(shape = (row,col,channel), name  = 'inputVGGPreTrain')
pt_vgg16 = preTrainModel(ptInput)

In [119]:
Mout = Flatten(name = 'flatten')(pt_vgg16)
Mout = Dense(1024, activation='relu', name = 'FC1_4096')(Mout)
Mout = Dense(1024, activation='relu', name = 'FC1_512')(Mout)

In [120]:
numd_SM = Dense(digLen,    activation='softmax',name = 'num')(Mout)
dig1_SM = Dense(numDigits, activation='softmax',name = 'dig1')(Mout)
dig2_SM = Dense(numDigits, activation='softmax',name = 'dig2')(Mout)
dig3_SM = Dense(numDigits, activation='softmax',name = 'dig3')(Mout)
dig4_SM = Dense(numDigits, activation='softmax',name = 'dig4')(Mout)
numB_SM = Dense(2, activation='softmax',name = 'nC')(Mout)
out = [numd_SM, dig1_SM ,dig2_SM, dig3_SM, dig4_SM,numB_SM] #numd_SM

In [121]:
vggPreTrain = keras.Model(inputs = ptInput, outputs = out)

vggPreTrain.compile(loss = 'sparse_categorical_crossentropy', #ceLoss ,
                    optimizer= optim,
                    metrics=  ['accuracy']) #[])
vggPreTrain.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputVGGPreTrain (InputLayer)   (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
vgg16 (Model)                   multiple             14714688    inputVGGPreTrain[0][0]           
__________________________________________________________________________________________________
flatten (Flatten)               (None, 2048)         0           vgg16[1][0]                      
__________________________________________________________________________________________________
FC1_4096 (Dense)                (None, 1024)         2098176     flatten[0][0]                    
__________________________________________________________________________________________________
FC1_512 (D

In [123]:
callback = []
checkpointer = keras.callbacks.ModelCheckpoint(filepath='saved_models/VGGPreTrained.classifier.hdf5',
                                               monitor='loss',
                                               save_best_only=True,
                                               verbose=2)

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = 'loss',
                                              factor = 0.1,
                                              verbose = 1,
                                              patience= 4,
                                              cooldown= 1,
                                              min_lr = 0.0001)
es = keras.callbacks.EarlyStopping(monitor= 'loss',
                                   min_delta=0.000001,
                                   patience=5,
                                   verbose=1,
                                   mode='auto')
callback.append(es)
callback.append(checkpointer)
callback.append(reduce_lr)

In [125]:
%%time
vggHistory = vggPreTrain.fit(x = X_train,
                             y = list(y_train.T),
                             batch_size = batch_size,
                             epochs=epochs,
                             verbose=1,
                             shuffle = True,
                             validation_data = (X_val, list(y_val.T)),
                             callbacks= callback)

print(vggHistory.history.keys())
modName = 'vgg16_PreTrain'
# list all data in history
print(vggHistory.history.keys())

Train on 69076 samples, validate on 17269 samples
Epoch 1/50
69076/69076 [==============================] - 88s 1ms/step - loss: 2.3698 - num_loss: 0.2870 - dig1_loss: 0.7737 - dig2_loss: 0.8096 - dig3_loss: 0.3982 - dig4_loss: 0.0864 - nC_loss: 0.0149 - num_acc: 0.8856 - dig1_acc: 0.7285 - dig2_acc: 0.7145 - dig3_acc: 0.8970 - dig4_acc: 0.9849 - nC_acc: 0.9940 - val_loss: 2.0318 - val_num_loss: 0.2093 - val_dig1_loss: 0.6195 - val_dig2_loss: 0.7659 - val_dig3_loss: 0.3629 - val_dig4_loss: 0.0726 - val_nC_loss: 0.0016 - val_num_acc: 0.9184 - val_dig1_acc: 0.7934 - val_dig2_acc: 0.7268 - val_dig3_acc: 0.8959 - val_dig4_acc: 0.9865 - val_nC_acc: 1.0000

Epoch 00001: loss improved from inf to 2.36981, saving model to saved_models/VGGPreTrained.classifier.hdf5
Epoch 2/50
69076/69076 [==============================] - 83s 1ms/step - loss: 1.6196 - num_loss: 0.1428 - dig1_loss: 0.4816 - dig2_loss: 0.6163 - dig3_loss: 0.3084 - dig4_loss: 0.0695 - nC_loss: 9.2504e-04 - num_acc: 0.9486 - dig1_a

69076/69076 [==============================] - 85s 1ms/step - loss: 0.2303 - num_loss: 0.0194 - dig1_loss: 0.0656 - dig2_loss: 0.0779 - dig3_loss: 0.0471 - dig4_loss: 0.0204 - nC_loss: 1.5151e-05 - num_acc: 0.9933 - dig1_acc: 0.9796 - dig2_acc: 0.9761 - dig3_acc: 0.9847 - dig4_acc: 0.9928 - nC_acc: 1.0000 - val_loss: 0.4850 - val_num_loss: 0.0547 - val_dig1_loss: 0.1238 - val_dig2_loss: 0.1733 - val_dig3_loss: 0.0995 - val_dig4_loss: 0.0333 - val_nC_loss: 5.2572e-04 - val_num_acc: 0.9848 - val_dig1_acc: 0.9654 - val_dig2_acc: 0.9530 - val_dig3_acc: 0.9740 - val_dig4_acc: 0.9910 - val_nC_acc: 0.9999

Epoch 00012: loss improved from 0.30518 to 0.23032, saving model to saved_models/VGGPreTrained.classifier.hdf5
Epoch 13/50
69076/69076 [==============================] - 86s 1ms/step - loss: 0.2095 - num_loss: 0.0179 - dig1_loss: 0.0597 - dig2_loss: 0.0726 - dig3_loss: 0.0414 - dig4_loss: 0.0179 - nC_loss: 1.5698e-05 - num_acc: 0.9941 - dig1_acc: 0.9812 - dig2_acc: 0.9765 - dig3_acc: 0.9860

# vgg_pretrain_result

In [171]:
%load_ext autoreload
%autoreload 2
from utils.model_evaluation_utils import evaluate_model_performance

evaluate_model_performance(fitted_model=vggHistory, 
                           model_name = 'pretrain_vgg16',
                           data=data,
                           model=vggPreTrain
                          )

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [173]:
with open('saved_models/vggPreTrain.pkl', 'wb') as f:
    pickle.dump(vggPreTrain, f)

In [174]:
with open('saved_models/vggHistory.pkl', 'wb') as f:
    pickle.dump(vggHistory, f)

#  detection